Librerias necesarias

In [13]:
import pandas as pd
import os 
import random
from faker import Faker
from datetime import datetime, timedelta
from IPython.display import display, Markdown

# Generacion de base de datos para clientes



In [15]:
fake = Faker('es_CL')

def generar_cliente():
  nombre = fake.name()
  correo = fake.email()
  telefono = fake.phone_number()
  ciudad = fake.city()

  return {"Nombre": nombre, "Ciudad": ciudad, "Correo": correo, "Telefono": telefono}

cantidad_clientes = 20

cliente = [generar_cliente() for _ in range (cantidad_clientes)]

df = pd.DataFrame(cliente)

# Carpeta para almacenar las bases de datos generadas
carpeta_database = 'database'

# Crear una carpeta si esta no existe
if not os.path.exists(carpeta_database):
  os.makedirs(carpeta_database)

# Path para guardar el archivo
path_info_clientes = os.path.join(carpeta_database, 'info_clientes.csv')

# Guardar el archivo
df.to_csv(path_info_clientes)

# Visualizar en markdown
display(Markdown(df.to_markdown()))

|    | Nombre                            | Ciudad               | Correo                      | Telefono        |
|---:|:----------------------------------|:---------------------|:----------------------------|:----------------|
|  0 | Joan Zúñiga Bravo                 | Calbuco              | esaez@example.org           | +56 600 670 333 |
|  1 | Enzo Rodrigo Martínez Cruz        | Cabo de Hornos       | wvera@example.com           | +56 51 343 3015 |
|  2 | Emmanuel Lagos-Troncoso           | San Rafael           | diazjuan@example.net        | +56 2 2999 7593 |
|  3 | Martín Rodríguez Salinas          | Taltal               | gustavoletelier@example.org | +56 800 121 725 |
|  4 | Loreto María Zamorano Vega        | Cauquenes            | bgonzalez@example.org       | +56 2 3564 1176 |
|  5 | Angie Romero                      | San Juan de la Costa | rmontero@example.org        | +56 2 2229 2498 |
|  6 | Angelina Ana Rodríguez Escobar    | Llaillay             | vcasanova@example.net       | +56 2 2713 1475 |
|  7 | César Víctor Muñoz Saavedra       | San Clemente         | gonzalezfranco@example.org  | +56 9 8800 2176 |
|  8 | Sergio Luis Cáceres Lobos         | Melipilla            | ceciliaortiz@example.org    | +56 800 440 135 |
|  9 | David Carlos Riquelme Pacheco     | General Lagos        | dmoyano@example.com         | +56 2 3723 9383 |
| 10 | Raquel Vergara Henríquez          | Puente Alto          | nunezjacqueline@example.com | +56 35 339 4038 |
| 11 | Vicente Alfonso Morales Salas     | Cunco                | mariazambrano@example.com   | +56 9 6341 0956 |
| 12 | Pascal Franchesca Catalán Ahumada | María Elena          | mario37@example.org         | +56 800 702 909 |
| 13 | Nancy Maribel Leiva Quintana      | Calama               | marcelina97@example.com     | +56 72 296 5309 |
| 14 | Juan Ayala Díaz                   | Alhué                | rosasepulveda@example.com   | +56 51 299 7304 |
| 15 | Carmen Salazar García             | Diego de Almagro     | hectortapia@example.org     | +56 67 268 2556 |
| 16 | Sebastián Cristian Cárcamo Campos | San Vicente          | arriagadaregina@example.com | +56 65 221 0105 |
| 17 | Carmen Georgina Pinto Escobar     | Coihaique            | arnaldomartinez@example.net | +56 55 368 4542 |
| 18 | Margarita Carrillo                | La Estrella          | jmorales@example.net        | +56 2 3846 9452 |
| 19 | Aron Aros Pérez                   | Los Álamos           | ssanhueza@example.org       | +56 2 3287 8255 |

# Generacion de base de datos para inventario

In [12]:
# Listas de tipos de productos y sabores
tipos_productos = ['Torta', 'Queque', 'Brownie', 'Rosca', 'Dona']
sabores = ['Chocolate', 'Frutilla', 'Vainilla', 'Caramelo', 'Nuez']

In [6]:
def generar_producto():
  nombre_producto = fake.unique.word()
  precio = fake.random_int(min=1000, max=10000)
  tipo_producto = random.choice(tipos_productos)
  sabor = random.choice(sabores)
  descripcion = f"{tipo_producto} sabor {sabor}"

  return {"Nombre": nombre_producto, "Descripcion": descripcion,"Precio": precio}

num_productos = 20

producto = [generar_producto() for _ in range (num_productos)]

df = pd.DataFrame(producto)

display(Markdown(df.to_markdown()))

|    | Nombre         | Descripcion            |   Precio |
|---:|:---------------|:-----------------------|---------:|
|  0 | maxime         | Rosca sabor Frutilla   |     1338 |
|  1 | tempora        | Rosca sabor Caramelo   |     4414 |
|  2 | odio           | Rosca sabor Vainilla   |     7185 |
|  3 | quis           | Brownie sabor Caramelo |     2408 |
|  4 | cum            | Rosca sabor Chocolate  |     6486 |
|  5 | quod           | Torta sabor Nuez       |     3497 |
|  6 | sunt           | Torta sabor Frutilla   |     1969 |
|  7 | repellat       | Brownie sabor Caramelo |     9994 |
|  8 | aut            | Torta sabor Vainilla   |     6000 |
|  9 | repellendus    | Dona sabor Vainilla    |     6781 |
| 10 | sequi          | Queque sabor Vainilla  |     7182 |
| 11 | reiciendis     | Torta sabor Nuez       |     9697 |
| 12 | ea             | Dona sabor Chocolate   |     2584 |
| 13 | necessitatibus | Dona sabor Nuez        |     1010 |
| 14 | voluptatem     | Dona sabor Vainilla    |     7872 |
| 15 | laudantium     | Brownie sabor Frutilla |     5957 |
| 16 | aspernatur     | Rosca sabor Frutilla   |     9171 |
| 17 | perferendis    | Brownie sabor Nuez     |     4863 |
| 18 | suscipit       | Rosca sabor Caramelo   |     7547 |
| 19 | pariatur       | Torta sabor Caramelo   |     4234 |

# Generacion de base de datos para ventas

## Leer base de datos de clientes
Esto es un paso necesario para porder asociar los clientes ya generados y de los cuales tenemos informacion, con los productos en inventario.

In [ ]:
df_clientes = pd.read_csv('info_clientes.csv')

nombres_clientes = df_clientes['Nombre']

In [ ]:
# Establecer un rango realista para las fechas de compras
fecha_actual = datetime.now()

inicio_rango = fecha_actual - timedelta (days = 180)

Se genera una tabla que contenga una fecha realista para la demanda del producto, un nombre de cliente extraido de la base de datos y un tipo de producto que desee. Cabe destacar que de esta forma y usando la funcion 'random.choice' es que puede existir clientes que demanden mas de un tipo de producto, lo cual es completamente valido y cumple con una situacion realista. 

In [10]:
def generar_demanda():
  fecha = fake.date_between_dates(date_start=inicio_rango, date_end=fecha_actual)
  nombre_cliente = random.choice(nombres_clientes)

  return {"Fecha": fecha, "Nombre": nombre_cliente, }

num_demanda_clientes = 20

demanda = [generar_demanda() for _ in range (num_demanda_clientes)]

df = pd.DataFrame(demanda)

display(Markdown(df.to_markdown()))

|    | Fecha      | Nombre                           |
|---:|:-----------|:---------------------------------|
|  0 | 2023-06-14 | Santiago Cubillos Piña           |
|  1 | 2023-10-25 | Héctor Pedro Miranda Abarca      |
|  2 | 2023-08-13 | Irene Laura Sepúlveda Delgado    |
|  3 | 2023-10-26 | Margarita Francisca Gómez Muñoz  |
|  4 | 2023-11-16 | Laura Jacqueline Roa Vera        |
|  5 | 2023-09-13 | María Eva Castillo Ruiz          |
|  6 | 2023-07-07 | Ignacia Anahí Parraguez Martínez |
|  7 | 2023-06-05 | Rafaella Doris Torres Manríquez  |
|  8 | 2023-06-15 | Irene Laura Sepúlveda Delgado    |
|  9 | 2023-07-04 | Martín Vidal López               |
| 10 | 2023-11-12 | Raquel Martínez Romero           |
| 11 | 2023-09-14 | Francisco Gustavo Jofré Gajardo  |
| 12 | 2023-09-04 | Margarita Francisca Gómez Muñoz  |
| 13 | 2023-11-07 | Ricardo Gabriel González Ávila   |
| 14 | 2023-08-16 | Héctor Pedro Miranda Abarca      |
| 15 | 2023-06-14 | Francisco Gustavo Jofré Gajardo  |
| 16 | 2023-10-08 | Héctor Pedro Miranda Abarca      |
| 17 | 2023-10-14 | Héctor Pedro Miranda Abarca      |
| 18 | 2023-10-25 | Ricardo Gabriel González Ávila   |
| 19 | 2023-11-16 | Rosario Leyton Pinto             |